In [1]:
import pandas as pd
URM_path = "data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                sep=",",
                                # header=None,
                                dtype={0: int, 1: int, 2: int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]
import scipy.sparse as sps

mapped_id, original_id = pd.factorize(URM_all_dataframe["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [4]:
from Data_manager.UserUtils import *
import pandas as pd
import numpy as np
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender


# Load the CSV file
file_path = 'submission_slim_rp3_item_13954.csv'
df = pd.read_csv(file_path)

URM_all = getURM_all()
users_few_iterations = URM_all.sum(axis=1)  
users_few_iterations = np.array(users_few_iterations).squeeze()
users_few_iterations = np.where((users_few_iterations < 2) * (users_few_iterations > 0))[0]

top = UserKNNCFRecommender(URM_all)
top.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)

item_index_to_original_id = item_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)

update_dict = {}
for user_id in users_few_iterations:
    recommendations = [item_index_to_original_id[index] for index in top.recommend(user_id, cutoff=10)]
    recommendations = " ".join(str(x) for x in recommendations)
    user_original = user_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)[user_id]
    update_dict[user_original] = recommendations



Similarity column 12638 (100.0%), 7563.27 column/sec. Elapsed time 1.67 sec


In [5]:
update_dict

{25: '51 7 15 4 18 85 2 107 10 14',
 31: '6 312 8 1531 192 322 70 245 14 78',
 53: '859 3970 720 910 254 3687 146 133 135 36',
 66: '1 202 4 2 81 15 75 8 6 88',
 80: '21 10305 630 653 274 4855 7110 2522 256 16542',
 105: '3 7 4 15 21 9 22 1 32 29',
 116: '15 101 7 1 3 6 4 2 8 34',
 120: '2 6 3 7 4 8 1 30 20 25',
 125: '15 101 7 1 3 6 2 4 8 34',
 161: '89 5 343 308 10 390 2 3 282 9',
 170: '35 450 224 184 361 11 558 288 319 4',
 186: '12 1 5 2 4 3 8 405 150 17',
 190: '2 3 4 1 6 8 112 15 11 10',
 195: '3 3565 3325 7 94 4 262 36 33 34',
 209: '2 3 6 4 26 1 7 9 5 49',
 222: '4 1 67 8 20 2 172 238 6 29',
 228: '4 1 3 8 2 7 20 61 19 74',
 238: '4 2 1 80 8 6 20 174 22 33',
 296: '4 8 2 26 6 72 7 1 20 14',
 304: '211 95 2 22 175 116 4 26 33 193',
 313: '4168 5728 1424 21198 15 8745 2591 4819 3687 419',
 335: '28 17 47 37 50 2 1 27 42 7',
 341: '7 2 3 380 6 422 56 949 41 30',
 353: '10 2 848 3 7 1 32 4 5 9',
 359: '153 285 14 59 213 6582 7474 8 26 3395',
 378: '4 2 6 3 8 7 38 19 24 9',
 452: '

In [6]:
# Update the item_list for the specified userIDs
for user_id, new_item_list in update_dict.items():
    df.loc[df['user_id'] == user_id, 'item_list'] = new_item_list

# Save the updated dataframe back to a CSV file
df.to_csv('updated_submission.csv', index=False)